In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Importing required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import InputLayer, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19

# Resizing the images
train_dir = '/kaggle/input/sarscov2-ctscan-dataset/COVID/non-COVID/Train'
test_dir = '/kaggle/input/sarscov2-ctscan-dataset/COVID/non-COVID/Test'

train_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_data = train_generator.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary' 
)

test_generator = ImageDataGenerator(rescale=1./255)
test_data = test_generator.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary'
)

# Creating Required Model
model = VGG19(include_top=False, weights='imagenet')

# Stopping the Layers from Training Again
for layer in model.layers:
    layer.trainable = False

# Creating Top Layer
input_layer = InputLayer(shape=(224, 224, 3))
x = Flatten()(input_layer.output)
output_layer = Dense(1, activation='sigmoid')(x)

# Adding the Top and Fully Connected Layers
model = Model(inputs=input_layer.input, outputs=output_layer)

# Compiling the Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fitting the Model
history = model.fit(train_data, epochs=5, steps_per_epoch=len(train_data))
